# **Projeto: Extração de Dados de Ações com a API Alpha Vantage**

A API **Alpha Vantage** fornece informações sobre cotações de preços de ações de algumas das maiores empresas do mundo. Este projeto realiza a extração, processamento e análise de dados financeiros dessas empresas.

## Empresas Monitoradas
- **Apple (AAPL)**
- **Google (GOOG)**
- **Microsoft (MSFT)**

---

## **Descrição do Dataset**

O dataset extraído contém os seguintes campos:

- **`open`**:  
  O preço de **abertura** da ação no dia. Representa o valor da primeira negociação realizada quando o mercado abriu naquele dia.

- **`high`**:  
  O preço mais **alto** atingido pela ação durante o dia. Representa o maior valor registrado pela ação enquanto o mercado estava aberto.

- **`low`**:  
  O preço mais **baixo** atingido pela ação durante o dia. Representa o menor valor registrado pela ação enquanto o mercado estava aberto.

- **`close`**:  
  O preço de **fechamento** da ação no final do dia. Representa o valor da última negociação realizada antes do mercado fechar.

- **`volume`**:  
  O **volume** de ações negociadas durante o dia. Indica o número total de ações compradas e vendidas naquele período.

- **`symbol`**:  
  O **símbolo** da ação. Identifica a empresa associada à ação (e.g., `AAPL` para Apple, `GOOG` para Google, `MSFT` para Microsoft).

---

## **Objetivos do Projeto**
1. Extrair dados históricos das ações das empresas monitoradas utilizando a API Alpha Vantage.
2. Consolidar os dados de todas as empresas em um único dataset.
3. Analisar o comportamento das ações, incluindo preços de abertura, máximos, mínimos, fechamentos e volumes de negociação.
4. Gerar insights sobre a performance das ações ao longo do tempo.

---

## **Tecnologias Utilizadas**
- **Python**: Linguagem principal para a extração e processamento de dados.
- **Bibliotecas**:
  - `requests`: Para acessar a API Alpha Vantage.
  - `pandas`: Para manipulação e análise dos dados.
  - `matplotlib` e `seaborn`: Para visualização dos dados (opcional).

---

## **Passos do Projeto**
1. **Configuração**: Obtenção da chave de API e definição dos símbolos das ações a serem monitoradas.
2. **Extração**: Coleta dos dados históricos das ações diretamente da API.
3. **Transformação**:
   - Normalização dos dados (renomeação de colunas e conversão de datas).
   - Consolidação de múltiplos datasets em um único DataFrame.
4. **Análise**: Geração de estatísticas descritivas e gráficos sobre o comportamento das ações.

---

## **Exemplo de Dados Extraídos**
| Date       | open    | high    | low     | close   | volume    | symbol |
|------------|---------|---------|---------|---------|-----------|--------|
| 2024-11-22 | 228.060 | 230.720 | 228.060 | 229.870 | 38168252  | AAPL   |
| 2024-11-21 | 228.880 | 230.155 | 225.710 | 228.520 | 42108327  | AAPL   |
| 2024-11-20 | 228.060 | 229.930 | 225.890 | 229.000 | 35169566  | AAPL   |
| 2024-11-19 | 226.980 | 230.160 | 226.660 | 228.280 | 36211774  | AAPL   |
| 2024-11-18 | 225.250 | 229.740 | 225.170 | 228.020 | 44686020  | AAPL   |




# Bibliotecas necessárias

In [31]:
import requests
from datetime import datetime
import pandas as pd
import matplotlib.pyplot as plt

In [40]:
import requests
import pandas as pd

api_key = "SY1X62CUITLR11BG"

# List of stock symbols
stocks = ["AAPL", "MSFT", "GOOG"]

# Function to fetch daily stock data
def fetch_stock_data(symbol):
    url = f"https://www.alphavantage.co/query?function=TIME_SERIES_DAILY&symbol={symbol}&outputsize=full&apikey={api_key}"
    response = requests.get(url)
    data = response.json()
    
    # Extracting stock data
    if 'Time Series (Daily)' in data:
        stock_data = data['Time Series (Daily)']
        df = pd.DataFrame.from_dict(stock_data, orient='index')
        df = df.rename(columns={
            '1. open': 'open',
            '2. high': 'high',
            '3. low': 'low',
            '4. close': 'close',
            '5. volume': 'volume'
        })
        df.index = pd.to_datetime(df.index) 
        
        # Add a column for the stock symbol
        df['symbol'] = symbol
        
        return df
    else:
        print(f"Error fetching data for {symbol}")
        return None

# List to store DataFrames
all_data = []




# Unindo todos os dataframes e convertendo colunas numéricas
Nossa lista contém todos os dados armazenados com as informações das empresas a partir disso podemos unir os dados para formar 1 único dataframe.

In [41]:
# Using map and lambda to collect data for each stock
all_data = list(
    map(lambda stock: fetch_stock_data(stock), stocks)
)

# Filter out None values
all_data = [data for data in all_data if data is not None]

# Convert numeric columns to float without altering 'symbol'
for i in range(len(all_data)):
    numeric_cols = all_data[i].columns.difference(['symbol'])
    all_data[i][numeric_cols] = all_data[i][numeric_cols].apply(pd.to_numeric, errors='coerce')

# Concatenate all DataFrames into a single DataFrame
complete_df = pd.concat(all_data)

# Display the combined data
print(complete_df.head())  # Displaying the first 5 rows

              open      high       low   close    volume symbol
2024-11-22  228.06  230.7199  228.0600  229.87  38168252   AAPL
2024-11-21  228.88  230.1550  225.7103  228.52  42108327   AAPL
2024-11-20  228.06  229.9300  225.8900  229.00  35169566   AAPL
2024-11-19  226.98  230.1600  226.6600  228.28  36211774   AAPL
2024-11-18  225.25  229.7400  225.1700  228.02  44686020   AAPL


In [26]:
complete_df['difference_price'] = complete_df['open'] - complete_df['close']

In [42]:
complete_df.size

91794

## **Variação diária de preços (high - low)**
Analisar a variação diária é muito útil para identificar a volatilidade de cada ação diariamente.

In [44]:


# Calculando a variação diária de preços
complete_df['daily_range'] = complete_df['high'] - complete_df['low']

# Exibindo os dias com maior volatilidade
top_volatility = complete_df.nlargest(5, 'daily_range')
print("Dias com maior variação diária:")
print(top_volatility[['symbol', 'daily_range', 'high', 'low']])


Dias com maior variação diária:
           symbol  daily_range       high        low
2021-10-27   GOOG     184.3100  2982.3600  2798.0500
2022-02-24   GOOG     165.4500  2660.7400  2495.2900
2022-07-05   GOOG     156.0615  2281.0516  2124.9901
2022-05-04   GOOG     148.0900  2462.8600  2314.7700
2022-02-03   GOOG     138.2544  2982.3544  2844.1000


## **Volume médio por ação**

**Insight:**  
O volume médio ajuda a entender a liquidez de cada ação. Ações com maior volume médio indicam maior interesse do mercado, enquanto volumes baixos podem sinalizar menor liquidez.

In [ ]:
average_volume = complete_df.groupby('symbol')['volume'].mean()

print("Volume médio por ação:")
print(average_volume)

Volume médio por ação:
symbol
AAPL    3.317143e+07
GOOG    6.453363e+06
MSFT    4.508674e+07
Name: volume, dtype: float64


## **Média de preço de fechamento por ano**
Isso pode revelar tendências gerais de valorização ou desvalorização ao longo do tempo.

In [47]:
# Adicionando uma coluna para o ano
complete_df['year'] = complete_df.index.year

# Calculando o preço médio de fechamento por ano e por ação
avg_close_per_year = complete_df.groupby(['symbol', 'year'])['close'].mean()

print("Preço médio de fechamento por ano:")
print(avg_close_per_year)

Preço médio de fechamento por ano:
symbol  year
AAPL    1999     96.859302
        2000     71.748929
        2001     20.219113
        2002     19.139524
        2003     18.543349
                   ...    
MSFT    2020    193.026126
        2021    275.940774
        2022    268.917092
        2023    313.950980
        2024    418.492907
Name: close, Length: 63, dtype: float64


## **Correlação entre preço de abertura e fechamento**
Pode indicar o quão próximas são as aberturas e fechamentos diários de uma ação.

In [48]:
# Correlação entre preço de abertura e fechamento para cada ação
correlations = complete_df.groupby('symbol').apply(
    lambda x: x['open'].corr(x['close'])
)

print("Correlação entre abertura e fechamento por ação:")
print(correlations)


Correlação entre abertura e fechamento por ação:
symbol
AAPL    0.999742
GOOG    0.999731
MSFT    0.999837
dtype: float64


C:\Users\geron\AppData\Local\Temp\ipykernel_18232\2971034999.py:2: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  correlations = complete_df.groupby('symbol').apply(


## **Análise de tendência com médias móveis**
Uma média móvel ajuda a suavizar os dados para identificar tendências.

In [49]:
# Calculando uma média móvel de 7 dias para o preço de fechamento
complete_df['7_day_moving_avg'] = complete_df.groupby('symbol')['close'].transform(lambda x: x.rolling(window=7).mean())

# Exibindo as primeiras linhas com a nova coluna
print("Média móvel de 7 dias adicionada:")
print(complete_df[['symbol', 'close', '7_day_moving_avg']].head(10))


Média móvel de 7 dias adicionada:
           symbol   close  7_day_moving_avg
2024-11-22   AAPL  229.87               NaN
2024-11-21   AAPL  228.52               NaN
2024-11-20   AAPL  229.00               NaN
2024-11-19   AAPL  228.28               NaN
2024-11-18   AAPL  228.02               NaN
2024-11-15   AAPL  225.00               NaN
2024-11-14   AAPL  228.22        228.130000
2024-11-13   AAPL  225.12        227.451429
2024-11-12   AAPL  224.23        226.838571
2024-11-11   AAPL  224.23        226.157143
